In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Hypothyroidism"
cohort = "GSE151158"

# Input paths
in_trait_dir = "../../input/GEO/Hypothyroidism"
in_cohort_dir = "../../input/GEO/Hypothyroidism/GSE151158"

# Output paths
out_data_file = "../../output/preprocess/Hypothyroidism/GSE151158.csv"
out_gene_data_file = "../../output/preprocess/Hypothyroidism/gene_data/GSE151158.csv"
out_clinical_data_file = "../../output/preprocess/Hypothyroidism/clinical_data/GSE151158.csv"
json_path = "../../output/preprocess/Hypothyroidism/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional analysis of non-fibrotic NAFLD progression"
!Series_summary	"Background & Aims:  Non-alcoholic steatohepatitis (NASH), a subtype of non-alcoholic fatty liver disease (NAFLD) that can lead to fibrosis, cirrhosis, and hepatocellular carcinoma, is characterized by hepatic inflammation.  Despite evolving therapies aimed to ameliorate inflammation in NASH, the transcriptional changes that lead to inflammation progression in NAFLD remain poorly understood.  The aim of this study is to define transcriptional changes in early, non-fibrotic NAFLD using a biopsy-proven non-fibrotic NAFLD cohort.  Methods:  We extracted RNA from liver tissue of 40 patients with biopsy-proven NAFLD based on NAFLD Activity Score (NAS) (23 with NAS ≤3, 17 with NAS ≥5) and 21 healthy controls and compared changes in expression of 594 genes involved in innate immune function.    Results:  Compared to healthy controls, NAFLD patients with NAS ≥5 had differential ex

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Determine if gene expression data is available
# Based on the background information, the study compares "expression of 594 genes involved in innate immune function"
# This confirms gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Trait (Hypothyroidism)
# From the sample characteristics dictionary, index 12 contains hypothyroidism data
trait_row = 12  # Hypothyroidism data is at key 12

# Define conversion function for trait data (binary)
def convert_trait(value):
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary (0 for N/no, 1 for Y/yes)
    if value.upper() == 'Y':
        return 1
    elif value.upper() == 'N':
        return 0
    else:
        return None

# 2.2 Age
# Age data is at key 1
age_row = 1

# Define conversion function for age (continuous)
def convert_age(value):
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to numeric value
    try:
        return float(value)
    except:
        return None

# 2.3 Gender
# Gender data is at key 2, labeled as "Sex"
gender_row = 2

# Define conversion function for gender (binary)
def convert_gender(value):
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary (0 for female, 1 for male)
    if value.upper() == 'F':
        return 0
    elif value.upper() == 'M':
        return 1
    else:
        return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Perform initial filtering and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we should extract clinical features
if trait_row is not None:
    # Assume clinical_data DataFrame is available from previous step
    try:
        # Extract relevant clinical features
        clinical_df = geo_select_clinical_features(
            clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the processed clinical data
        print("Preview of clinical data:")
        print(preview_df(clinical_df))
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the clinical data to CSV
        clinical_df.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except NameError:
        print("Error: clinical_data DataFrame not found. Please ensure it's available from previous steps.")


Preview of clinical data:
{'GSM4567420': [0.0, 53.0, 0.0], 'GSM4567421': [0.0, 40.0, 1.0], 'GSM4567422': [0.0, 51.0, 0.0], 'GSM4567423': [0.0, 36.0, 0.0], 'GSM4567424': [0.0, 44.0, 0.0], 'GSM4567425': [1.0, 60.0, 1.0], 'GSM4567426': [0.0, 31.0, 1.0], 'GSM4567427': [0.0, 41.0, 0.0], 'GSM4567428': [0.0, 55.0, 1.0], 'GSM4567429': [0.0, 15.0, 0.0], 'GSM4567430': [1.0, 57.0, 0.0], 'GSM4567431': [1.0, 56.0, 0.0], 'GSM4567432': [0.0, 34.0, 1.0], 'GSM4567433': [0.0, 43.0, 1.0], 'GSM4567434': [0.0, 49.0, 0.0], 'GSM4567435': [0.0, 55.0, 0.0], 'GSM4567436': [0.0, 52.0, 1.0], 'GSM4567437': [0.0, 35.0, 0.0], 'GSM4567438': [0.0, 35.0, 0.0], 'GSM4567439': [0.0, 40.0, 1.0], 'GSM4567440': [0.0, 34.0, 1.0], 'GSM4567441': [0.0, 42.0, 0.0], 'GSM4567442': [1.0, 53.0, 0.0], 'GSM4567443': [0.0, 33.0, 0.0], 'GSM4567444': [0.0, 31.0, 0.0], 'GSM4567445': [1.0, 57.0, 0.0], 'GSM4567446': [0.0, 42.0, 0.0], 'GSM4567447': [0.0, 48.0, 0.0], 'GSM4567448': [0.0, 47.0, 0.0], 'GSM4567449': [0.0, 51.0, 0.0], 'GSM4567450':

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:
Successfully extracted gene data with 618 rows
First 20 gene IDs:
Index(['ABCB1', 'ABCF1', 'ABL1', 'ADA', 'AHR', 'AICDA', 'AIRE', 'ALAS1', 'APP',
       'AREG', 'ARG1', 'ARG2', 'ARHGDIB', 'ATG10', 'ATG12', 'ATG16L1', 'ATG5',
       'ATG7', 'ATM', 'B2M'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers in the gene expression data 
# The first 20 gene IDs shown are human gene symbols (e.g., ABCB1, ABL1, ADA, etc.)
# These are standardized human gene symbols that don't need to be mapped to other identifiers

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols in the expression data...")
try:
    # If previous steps have already loaded gene_data
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Save normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    print(f"Normalized gene data shape: {normalized_gene_data.shape}")
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Link clinical and genetic data
if is_gene_available and 'clinical_df' in locals():
    print("\nLinking clinical and genetic data...")
    try:
        # Transpose normalized gene data to match clinical data format
        gene_data_t = normalized_gene_data.T
        
        # Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data_t)
        print(f"Linked data shape: {linked_data.shape}")
        
        # 3. Handle missing values systematically
        print("\nHandling missing values...")
        linked_data = handle_missing_values(linked_data, trait)
        print(f"After handling missing values, data shape: {linked_data.shape}")
        
        # 4. Determine whether features are biased
        print("\nChecking for bias in features...")
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        
        # 5. Final validation and save metadata
        print("\nPerforming final validation...")
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=is_biased,
            df=linked_data,
            note=""
        )
        
        # 6. Save the linked data if usable
        if is_usable:
            # Create directory if it doesn't exist
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            
            # Save linked data
            linked_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print(f"Dataset not usable for {trait} association studies. Data not saved.")
    
    except Exception as e:
        print(f"Error in data linking or processing: {e}")
        is_usable = False
else:
    print("Cannot link data: either gene data or clinical data is unavailable.")
    # Create a minimal dataframe for validation purposes
    import pandas as pd
    linked_data = pd.DataFrame({trait: [None]})
    
    # Perform final validation with appropriate flags
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Not relevant since data isn't usable
        df=linked_data,
        note="Failed to link gene and clinical data."
    )
    print(f"Dataset usability: {is_usable}")

Normalizing gene symbols in the expression data...
Normalized gene data saved to ../../output/preprocess/Hypothyroidism/gene_data/GSE151158.csv
Normalized gene data shape: (583, 66)

Linking clinical and genetic data...
Linked data shape: (649, 69)

Handling missing values...
After handling missing values, data shape: (0, 2)

Checking for bias in features...
Quartiles for 'Hypothyroidism':
  25%: nan
  50% (Median): nan
  75%: nan
Min: nan
Max: nan
The distribution of the feature 'Hypothyroidism' in this dataset is fine.

Quartiles for 'Age':
  25%: nan
  50% (Median): nan
  75%: nan
Min: nan
Max: nan
The distribution of the feature 'Age' in this dataset is fine.


Performing final validation...
Abnormality detected in the cohort: GSE151158. Preprocessing failed.
A new JSON file was created at: ../../output/preprocess/Hypothyroidism/cohort_info.json
Dataset not usable for Hypothyroidism association studies. Data not saved.
